In [11]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [12]:
# 1. 生成数据集
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)
features[:6], labels[:6] # 查看前6个数据样本

(tensor([[-0.2845, -0.0551],
         [ 1.2715, -0.9031],
         [ 1.3642,  0.2758],
         [ 0.7112,  0.4987],
         [ 0.5876, -1.1839],
         [ 0.1183,  0.7827]]),
 tensor([[3.8087],
         [9.8030],
         [5.9841],
         [3.9253],
         [9.4073],
         [1.7928]]))

In [13]:
# 2. 读取数据集
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [14]:
next(iter(data_iter))

[tensor([[ 0.4849,  2.9266],
         [ 0.6156, -0.0987],
         [-0.3601, -0.9089],
         [ 0.9055,  0.3124],
         [ 0.3484,  0.2103],
         [ 0.2050,  1.3840],
         [-1.9179,  0.3253],
         [ 0.9102,  0.6522],
         [ 2.3057, -0.4850],
         [ 0.2682,  1.1739]]),
 tensor([[-4.7899],
         [ 5.7621],
         [ 6.5678],
         [ 4.9519],
         [ 4.1766],
         [-0.0846],
         [-0.7305],
         [ 3.8050],
         [10.4569],
         [ 0.7450]])]

In [15]:
# 3. 定义模型
## nn是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1)) # 2个输入特征，1个输出

In [16]:
net[0].weight.data.normal_(0, 0.01) # 初始化权重
net[0].bias.data.fill_(0) # 初始化偏差

tensor([0.])

In [17]:
loss = nn.MSELoss() # 均方误差损失函数

In [18]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03) # 随机梯度下降优化器

In [22]:
# 4. 训练模型
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        """
        其实按照直觉来说，这里应该是先反向传播，再更新梯度，最后清零梯度
        而之所以先清零是确保无论上一轮循环发生了什么，或者循环外面有没有不小心留下的梯度，这轮一开始首先清零
        这也是一种防御性编程的体现

        # 这也是可以的！
        l.backward()        # 1. 算梯度
        trainer.step()      # 2. 用梯度更新参数 (此时梯度还在)
        trainer.zero_grad() # 3. 清理梯度，为下一轮做准备
        """
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000093
epoch 2, loss 0.000093
epoch 3, loss 0.000092


In [23]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([ 0.0004, -0.0007])
b的估计误差： tensor([0.0004])
